# Phase 3: Training and Testing for $t=2$
* Models : SVM, MLP, LightGBM, XGBoost, CatBoost

## Setup

First, let's import a few common modules.

In [20]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
import random
seed_val = 43
np.random.seed(seed_val)
random.seed(seed_val)


## Data Loading

In [21]:
seed = 42
df = pd.read_csv(f"datasets/train_set_{seed}_t_1.csv")

In [22]:
X = df.drop(['is_drop'], axis=1)
y = df['is_drop'].copy()

## Data Transformation

Label encoding for categorical features

In [23]:
def sex_mapping(sex):
    sex_map = {'M': 0, 'F': 1}
    return sex_map[sex]

def seme_mapping(seme): # semester (Spring/Fall)
    seme_map = {1: 0, 2: 1}
    return seme_map[seme]

def college_mapping(college):
    college_map = {'TH': 0, 'HS': 1, 'BZ': 2, 'HT': 3, 'EG': 4, 'SW': 5, 'AT': 6}
    return college_map[college]


In [24]:
def label_encoding(data):
    data['sex'] = data['sex'].map(sex_mapping)
    data['semester'] = data['semester'].map(seme_mapping)
    data['college'] = data['college'].map(college_mapping)

In [25]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_attrs = ['grade', 'gpa_last_seme', 'credits_last_seme', 'credits_tot', 'n_seme', 'years_since']
cat_attrs = ['semester', 'sex', 'adm_unit', 'nation', 'in_capa', 'college', 'leave']

full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', OneHotEncoder(), cat_attrs)
])

cat_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', 'passthrough', cat_attrs)
])

X_t = full_pipeline.fit_transform(X)    # for SVM and MLP with one-hot encoding
X_cat = X.copy()    # for CatBoost without encoding
label_encoding(X)   # for XGBoost and LightGBM with label encoding
X_xgb = cat_pipeline.fit_transform(X)
X_cat = cat_pipeline.fit_transform(X) # for CatBoost

X = pd.DataFrame(X_t, columns=full_pipeline.get_feature_names_out())        # for SVM and MLP
X_xgb = pd.DataFrame(X_xgb, columns=cat_pipeline.get_feature_names_out())   # for XGBoost and LightGBM
X_cat = pd.DataFrame(X_cat, columns=cat_pipeline.get_feature_names_out())   # for CatBoost
cat_indices = [6, 7, 8, 9, 10, 11, 12]


In [26]:
# Typesetting for LGBM
num_cols = ['num__grade', 'num__gpa_last_seme', 'num__credits_last_seme', 'num__credits_tot', 'num__n_seme', 'num__years_since']
cat_cols = list(set(X_cat.columns) - set(num_cols))

X_cat[num_cols] = X_cat[num_cols].astype(float)
X_cat[cat_cols] = X_cat[cat_cols].astype(int)
X_xgb[num_cols] = X_xgb[num_cols].astype(float)
X_xgb[cat_cols] = X_xgb[cat_cols].astype(int)

In [27]:
X_cat.rename(columns={'num__grade': 'year', 'num__gpa_last_seme': 'gpa_last', 'num__credits_last_seme': 'credits_last',
                  'num__credits_tot': 'credits_tot', 'num__n_seme': 'n_semesters', 'num__years_since': 'years_since',
                  'cat__semester': 'release', 'cat__sex': 'sex', 'cat__adm_unit': 'adm_type', 'cat__nation': 'nation',
                  'cat__in_capa': 'in_capa', 'cat__college': 'college', 'cat__leave': 'status'}, inplace=True)

## 5-fold CV and Threshold Optimization


In [28]:
from sklearn.model_selection import cross_val_predict, cross_validate

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [29]:

svm_clf = SVC(kernel='rbf', C=10, gamma=0.1, random_state=42)
mlp_clf = MLPClassifier(max_iter=10_000, hidden_layer_sizes=[10,10,10], random_state=43)
xgb_clf = XGBClassifier(max_depth=4, n_estimators=60)
lgb_clf = LGBMClassifier(max_depth=8, n_estimators=90)
cat_clf = CatBoostClassifier(cat_features=cat_indices, max_depth=6, n_estimators=1000)

In [30]:
from sklearn.metrics import (
    precision_recall_curve, recall_score, precision_score, f1_score,
    average_precision_score, roc_auc_score, matthews_corrcoef
)
from pprint import pprint

# function to evaluate a classifier with 5-fold CV with "thereshold optimization".
def run_model(clf, X_train, y_train): 
    res = cross_validate(clf, X_train, y_train, cv=5, scoring=('average_precision','roc_auc',  'recall', 'precision', 'f1', 'matthews_corrcoef'))

    auc = res['test_roc_auc'].mean()
    ap = res['test_average_precision'].mean()
    recall_no = res['test_recall'].mean()
    preci_no = res['test_precision'].mean()
    f1_no = res['test_f1'].mean()
    mcc_no = res['test_matthews_corrcoef'].mean()


    # Threshold optimization
    if (type(clf).__name__ == 'SVC'):
            y_score = cross_val_predict(clf, X_train, y_train, cv=5, method='decision_function')
    else: 
        y_score = cross_val_predict(clf, X_train, y_train, cv=5, method='predict_proba')[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_train, y_score)

    # Maximizing F1-score
    f1_val = (2 * precision * recall) / (precision + recall)
    f1_val[np.isinf(f1_val)] = np.nan

    i = np.nanargmax(f1_val)
    best_thre = thresholds[i]

    y_pred = (y_score >= best_thre)
    recall_opt = recall_score(y_train, y_pred)
    preci_opt = precision_score(y_train, y_pred)
    f1_opt = f1_score(y_train, y_pred)
    mcc_opt = matthews_corrcoef(y_train, y_pred)

    res = {'ap': ap, 'auc': auc, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc': mcc_no, 'recall(opt)': recall_opt, 'precision(opt)': preci_opt,
           'f1(opt)': f1_opt, 'mcc(opt)': mcc_opt, 'threshold': best_thre}
    return res

Run the evaluation and optimize the threshold for each model.

In [31]:
names = ['svm', 'mlp', 'lgb', 'xgb', 'cat']

cvres_svm = {}
cvres_mlp = {}
cvres_xgb = {}
cvres_lgb = {}
cvres_cat = {}

for name in names:
    clf = globals()[f'{name}_clf']
    if (name == 'cat'):
        globals()[f'cvres_{name}'] = run_model(clf, X_cat, y)
    elif (name in ['xgb', 'lgb']):
        globals()[f'cvres_{name}'] = run_model(clf, X_xgb, y)
    else:
        globals()[f'cvres_{name}'] = run_model(clf, X, y)
    

[LightGBM] [Info] Number of positive: 958, number of negative: 23872
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000811 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 493
[LightGBM] [Info] Number of data points in the train set: 24830, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.038582 -> initscore=-3.215614
[LightGBM] [Info] Start training from score -3.215614
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 958, number of negative: 23872
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of te

Results of 5-fold CV on the training set.

In [32]:
for name in names:
    clf = globals()[f'{name}_clf']
    print("model: ", type(clf).__name__)
    pprint(globals()[f'cvres_{name}'], sort_dicts=False)
    print()

model:  SVC
{'ap': 0.41964922400328686,
 'auc': 0.8696564505115146,
 'recall': 0.22708507670850767,
 'precision': 0.6549187083201803,
 'f1': 0.3360017055080058,
 'mcc': 0.37171098447065,
 'recall(opt)': 0.4749582637729549,
 'precision(opt)': 0.4462745098039216,
 'f1(opt)': 0.46016983420946217,
 'mcc(opt)': 0.43802459926246473,
 'threshold': -0.7867285755893446}

model:  MLPClassifier
{'ap': 0.4671781372233886,
 'auc': 0.9239680475086093,
 'recall': 0.30548117154811716,
 'precision': 0.6068318273170515,
 'f1': 0.4050753556092116,
 'mcc': 0.4143250487052567,
 'recall(opt)': 0.5325542570951586,
 'precision(opt)': 0.44490934449093444,
 'f1(opt)': 0.4848024316109423,
 'mcc(opt)': 0.46422059003592026,
 'threshold': 0.2263589028709953}

model:  LGBMClassifier
{'ap': 0.48793579778027574,
 'auc': 0.932120033600869,
 'recall': 0.2996966527196653,
 'precision': 0.6463559198298816,
 'f1': 0.408551555734078,
 'mcc': 0.42530240276888415,
 'recall(opt)': 0.5826377295492488,
 'precision(opt)': 0.44121

## Model Training and Test

In [33]:
# Now, we train the models on the training set
svm_clf.fit(X, y)
mlp_clf.fit(X, y)
xgb_clf.fit(X_xgb, y)
lgb_clf.fit(X_xgb, y)
cat_clf.fit(X_cat, y, cat_features=cat_indices)

[LightGBM] [Info] Number of positive: 1198, number of negative: 29840
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 494
[LightGBM] [Info] Number of data points in the train set: 31038, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.038598 -> initscore=-3.215196
[LightGBM] [Info] Start training from score -3.215196
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Learning rate set to 0.044666
0:	learn: 0.6115413	total: 190ms	remaining: 3m 9s
1:	learn: 0.5388768	total: 335ms	remaining: 2m 47s
2:	learn: 0.4761311	total: 502ms	remaining: 2m 46s
3:	learn: 0.4304463	total: 664ms	remaining: 2m 45s
4:	learn: 0.3837176	total: 827ms	remaining: 2m 44s
5:	learn: 

In [34]:
import joblib

names = ['svm', 'mlp', 'xgb', 'lgb', 'cat']

for name in names:
    if (name in ['xgb', 'lgb', 'cat']):
        model_dict = {
        'model': globals()[f'{name}_clf'],
        'pipeline': cat_pipeline,
        'result': globals()[f'cvres_{name}']
        }
    else:
        model_dict = {
        'model': globals()[f'{name}_clf'],
        'pipeline': full_pipeline,
        'result': globals()[f'cvres_{name}']
        }
    joblib.dump(model_dict, f'models/base_{name}_1_1.pkl')


In [35]:
test_set = pd.read_csv(f"datasets/test_set_{seed}_t_1.csv")

# Preprocessing test set
X_te = full_pipeline.transform(test_set)
label_encoding(test_set)
X_te_xgb = cat_pipeline.transform(test_set)
X_te_cat = cat_pipeline.transform(test_set)
y_te = test_set['is_drop'].copy()

X_te = pd.DataFrame(X_te, columns=full_pipeline.get_feature_names_out())
X_te_xgb = pd.DataFrame(X_te_xgb, columns=cat_pipeline.get_feature_names_out())
X_te_cat = pd.DataFrame(X_te_cat, columns=cat_pipeline.get_feature_names_out())

# Typesetting for LGBM
X_te_xgb[num_cols] = X_te_xgb[num_cols].astype(float)
X_te_xgb[cat_cols] = X_te_xgb[cat_cols].astype(int)

X_te_cat[num_cols] = X_te_cat[num_cols].astype(float)
X_te_cat[cat_cols] = X_te_cat[cat_cols].astype(int)

In [36]:
X_te_cat.rename(columns={'num__grade': 'year', 'num__gpa_last_seme': 'gpa_last', 'num__credits_last_seme': 'credits_last',
                  'num__credits_tot': 'credits_tot', 'num__n_seme': 'n_semesters', 'num__years_since': 'years_since',
                  'cat__semester': 'release', 'cat__sex': 'sex', 'cat__adm_unit': 'adm_type', 'cat__nation': 'nation',
                  'cat__in_capa': 'in_capa', 'cat__college': 'college', 'cat__leave': 'status'}, inplace=True)

In [37]:
# function to evaluate the performance with test set and given threshold

def test_model(clf, X_test, y_test, thre): 
    if (type(clf).__name__ == 'SVC'):
        y_score = clf.decision_function(X_test)
    else: y_score = clf.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_score)
    ap = average_precision_score(y_test, y_score)

    if (type(clf).__name__ == 'SVC'):
        y_pred = (y_score >= 0)
    else: y_pred = (y_score >= 0.5)
    recall_no = recall_score(y_test, y_pred)
    preci_no = precision_score(y_test, y_pred)
    f1_no = f1_score(y_test, y_pred)
    mcc_no = matthews_corrcoef(y_test, y_pred)

    # Optimizing threshold by maximizing F1-score

    y_pred = (y_score >= thre)
    recall_opt = recall_score(y_test, y_pred)
    preci_opt = precision_score(y_test, y_pred)
    f1_opt = f1_score(y_test, y_pred)
    mcc_opt = matthews_corrcoef(y_test, y_pred)


    res = {'auc': auc, 'ap': ap, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc_no': mcc_no, 'recall(opt)': recall_opt, 
           'precision(opt)': preci_opt, 'f1(opt)': f1_opt, 'mcc(opt)': mcc_opt,
           'threshold': thre}
    print("model: ", type(clf).__name__)
    pprint(res, sort_dicts=False)
    print()
    return res

In [38]:
# Now we test the six models with the test set and given thresholds.

test_svm = {}
test_mlp = {}
test_xgb = {}
test_lgb = {}
test_cat = {}

test_svm = test_model(svm_clf, X_te, y_te, cvres_svm['threshold'])
test_mlp = test_model(mlp_clf, X_te, y_te, cvres_mlp['threshold'])
test_lgb = test_model(lgb_clf, X_te_xgb, y_te, cvres_lgb['threshold'])
test_xgb = test_model(xgb_clf, X_te_xgb, y_te, cvres_xgb['threshold'])
test_cat = test_model(cat_clf, X_te_cat, y_te, cvres_cat['threshold'])


model:  SVC
{'auc': 0.853481033510759,
 'ap': 0.41389665093566685,
 'recall': 0.22945205479452055,
 'precision': 0.7204301075268817,
 'f1': 0.34805194805194806,
 'mcc_no': 0.39520054992608955,
 'recall(opt)': 0.476027397260274,
 'precision(opt)': 0.4542483660130719,
 'f1(opt)': 0.46488294314381273,
 'mcc(opt)': 0.44362768965680033,
 'threshold': -0.7867285755893446}

model:  MLPClassifier
{'auc': 0.9291648667842995,
 'ap': 0.4546019149165224,
 'recall': 0.3013698630136986,
 'precision': 0.6423357664233577,
 'f1': 0.41025641025641024,
 'mcc_no': 0.4260303045294885,
 'recall(opt)': 0.5513698630136986,
 'precision(opt)': 0.44846796657381616,
 'f1(opt)': 0.4946236559139785,
 'mcc(opt)': 0.4755373460034454,
 'threshold': 0.2263589028709953}

model:  LGBMClassifier
{'auc': 0.9312537980568592,
 'ap': 0.49240669917043994,
 'recall': 0.3184931506849315,
 'precision': 0.6503496503496503,
 'f1': 0.42758620689655175,
 'mcc_no': 0.44120031351581496,
 'recall(opt)': 0.5787671232876712,
 'precision(o